[View in Colaboratory](https://colab.research.google.com/github/tomaham/datasharing/blob/master/BibIndex_Auctoritates_colaboratory.ipynb)

In [0]:
#Auctoricates BibIndex script v1.2 (Colaboratory version)
###################################################################

#0] authenticate google user
#1] read google sheets from input params
#2] do the stuff
#3] prompt browser download dialog with result excel dataset
#4] generate html output
#5] prompt browser download dialog with result html file


#todoos
# validation of data? 
# warning for rows with error or non-standard values?
# better parametrisation of sorting?

#notes
#https://github.com/davisd/python-scriptures, http://www.davisd.com/python-scriptures/

In [0]:
!pip install gspread
!pip install openpyxl

  Running setup.py bdist_wheel for gspread ... - done
  Stored in directory: /content/.cache/pip/wheels/70/d4/87/c3e549f7548178be1cfb0f46f776919b979fdac98a9d270b5a
Successfully built gspread
    100% |████████████████████████████████| 174kB 6.1MB/s 
  Running setup.py bdist_wheel for openpyxl ... - \ done
  Stored in directory: /content/.cache/pip/wheels/b8/20/5b/d260e131180f4394eba48d97e238016f4bde050727fce79283
  Running setup.py bdist_wheel for et-xmlfile ... - done
  Stored in directory: /content/.cache/pip/wheels/2a/77/35/0da0965a057698121fc7d8c5a7a9955cdbfb3cc4e2423cad39
Successfully built openpyxl et-xmlfile


In [0]:
import pandas as pd
import numpy as np
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from google.colab import files

In [0]:
#scope=['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
#credentials = ServiceAccountCredentials.from_json_keyfile_name('DDDproject-f724d034f3f7.json', scope)
#gc = gspread.authorize(credentials)


#google colab autentication
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())


In [0]:
#input parameters for data

#google sheet needs to be shared with the account gsheet-auctoritates@dddproject-1086.iam.gserviceaccount.com
source_abbreviation_url = "https://docs.google.com/spreadsheets/d/1byVbRwnQ058--0ud1_NBt2xgIhuiV7LK1BmCm67CTxA/edit#gid=1657192709"#url tabulky
source_abbreviation_url_sheet = "AbbrevConverter" #nazev konkretniho sheetu

source_datasets_url = [["https://docs.google.com/spreadsheets/d/1byVbRwnQ058--0ud1_NBt2xgIhuiV7LK1BmCm67CTxA/edit#gid=1657192709","KatpramBiblicalQuotations"]]  # muze jich byt vic, budeme concatovat

projectName = "AUCT-"
for dt in source_datasets_url:
    projectName += dt[1]


#getting abbreviation sheet
wksabb = gc.open_by_url(source_abbreviation_url).worksheet(source_abbreviation_url_sheet)

#gettting data sheets
df_references = []
for source in source_datasets_url:
    wksdata = gc.open_by_url(source[0]).worksheet(source[1])
    df_references.append(pd.DataFrame(wksdata.get_all_records()))

In [0]:
#input parameters for operations

#reference string in references dataset
itemReferenceColumn = "Full_reference_Czech"
itemReferenceDenotation = itemReferenceColumn.split("_")[-1] #Czech


#params for understanding of reference parts NOT USED
signVerseBook = ','
signFromTo = '-'
signAnd = '.'




In [0]:
#params for output generation  partially USED

import time
ts = time.gmtime()
tstring = time.strftime("%Y-%m-%d-%H-%M-%S", ts)
# 2018-08-05-22-20-39
outputfilename = "output_" + projectName + "_" + tstring

classificationLevel1 = "název sloupce" # [hodnoty = Cathar, catholic] [řazení vždy abecedně-číselné, ale lze zadat parametr variantního sloupce, podle kterého sortovat]
classificationLevel2 = "název sloupce" # [hodnoty = OT/NT]
#...
classificationLevel6 = "název sloupce"



#Prototyp (Pokřtění ohněm):
#classificationLevel1 = ["Christianity_type_level1",""]  #cathar/catholic
#classificationLevel2 = ["Classification_testament",sortBy=["OT","NT"]] #NT/OT
#classificationLevel3 = ["Czech_full", sortBy=Book_ID]


#Prototyp (Pokřtění ohněm):  USED
classificationLevel1 = ["Christianity_type_level1",""]  #cathar/catholic
classificationLevel2 = ["Classification_testament",["OT","NT"]] #NT/OT
classificationLevel3 = ["Czech_full", "Book_ID"]


#NOT USED
rejoinSeparatedItems = True #[default=False] [“+” a “=” => u každého rozděleného citátu musí být do velké rozsekané tabulky vepsáno a) že to je rozdělená reference b) její typ (typ “+” nebo typ “=”)]

#NOT USED
distinguishCitationAllusion = False # [default=True]


#USED
leftDelimiterAllusion= '(' # [Určení, jak má v kódu HTML vypadat levý vymezovač aluze]
rightDelimiterAllusion= ')'
leftDelimiterCitation= ''
leftDelimiterCitation= ''

In [0]:
#cleaning dataset from empty rows/cells

df_abb = pd.DataFrame(wksabb.get_all_records()) #abbreviation dataset
df_abb_signatures = df_abb[df_abb["Book_ID"]  == "N/A"] #rows with punctiation symbols


#replace empty strings with "NaN"
df_abb = df_abb.applymap(lambda x: np.nan if isinstance(x, basestring) and (len(x) == 0 or x=="N/A") else x)
#remove rows which do not have set the type of testament (OT/NT)
df_abb = df_abb.dropna(subset=['Classification_testament'])


for idx, df in enumerate(df_references):
    #replace empty strings with "NaN"
    df = df.applymap(lambda x: np.nan if isinstance(x, basestring) and len(x) == 0 else x)
    #remove rows which do not have set the type of citations (ciation/allusion)
    df = df.dropna(subset=['Citation_or_allusion'])
    df_references[idx] = df
    
#df_abb_signatures

In [0]:
#df_references[0]

In [0]:
#enrichment of references dataset
#rtype >  single / multiple / mitem


def is_reference_multiple(teststring):
    possibles = ["+","=",";"]    
    
    if any(s in teststring for s in possibles):  #multiple reference type
        for sign in possibles:
            if (sign in teststring):
                found = sign
                
        return "multiple"+found
    else:
        return "single"  

def count_children_reference_multiple(row):        
    teststring = row[itemReferenceColumn]
    rowid = row.index
    possibles = ["+","=",";"]    
    
    if any(s in teststring for s in possibles):  #multiple reference type        
        for sign in possibles:
            if (sign in teststring):
                parts = teststring.split(sign)            
        count = len(parts)        
        return count
    else:
        return 1      

   
    

#new columns for composition manipulation
for idx, df in enumerate(df_references):
    df['Composite_type'] = df.apply(lambda row: is_reference_multiple(row[itemReferenceColumn]), axis=1)
    df['Composite_count'] = df.apply(lambda row: count_children_reference_multiple(row), axis=1)
    df['Composite_parent'] = df.index
    df['Composite_reference'] = df[itemReferenceColumn]
    
    #iterate over dataset and create children for multi items
    for index, row in df.iterrows():
        if (row['Composite_count'] > 1):
                                  
            parts = row[itemReferenceColumn].split(row['Composite_type'][-1])
            
            row['Composite_type'] = 'child'+ row['Composite_type'][-1]
            row['Composite_parent'] = index
            row['Composite_reference'] = ""
            
            for part in parts:
                row['Composite_reference'] = part.strip()           
                df.loc[len(df.index)]= row  #create child row
    
    
    df_references[idx] = df
    
#df_references[0]

In [0]:
#enrichment of references dataset


#book_indentity > determination of bible book for the reference
def determine_book_abb(row):
    
    if ("multiple" not in row['Composite_type']):
        
        string = row['Composite_reference'].split(" ")
        
        if (string[0].isdigit()): 
            book_abb = string[0] + " " + string[1]
        else:
            book_abb = string[0]        
        
        return book_abb
    else:
        return np.nan

#chapter determination    
def determine_chapter(row):
    
    if ("multiple" not in row['Composite_type']):
        
        string = row['Composite_reference'].split(" ")
        
        if (string[0].isdigit()): 
            book_abb = string[0] + " " + string[1]
        else:
            book_abb = string[0]        
        
        lookingforchapter = row['Composite_reference'].split(",")
        
        if (len(lookingforchapter) > 2):
            #raise ValueError('More chapter delimiters in composite reference: ' + row['Composite_reference'])
            print 'Warning','More chapter delimiters in composite reference: ',row['Composite_reference']
        
        chapter = lookingforchapter[0].replace(book_abb,"")
        
        return int(chapter)
    else:
        return np.nan

#verses determinantion      
def determine_verses(row):
    
    if ("multiple" not in row['Composite_type']):
        
        string = row['Composite_reference'].split(" ")
        
        if (string[0].isdigit()): 
            book_abb = string[0] + " " + string[1]
        else:
            book_abb = string[0]        
        
        lookingforverses = row['Composite_reference'].split(",")
        
        if (len(lookingforverses) > 2):
            #raise ValueError('A more chapter delimiters in composite reference: ' + row['Composite_reference'])
            print 'Warning','A more chapter delimiters in composite reference: ',row['Composite_reference']
        
        verses = lookingforverses[1]
        
        return verses
    else:
        return np.nan

#first verse determinantion 
def determine_firstverse(row):
    
    if ("multiple" not in row['Composite_type']):
        verses = row['Composite_reference_verses']
        if (" " in verses):
            verses = verses.split(" ")[0]
    
        if ("-" in verses):
            string = verses.split("-") 
            verse = string[0]
            
            if ("." in verse):
                string = verse.split(".")
                if (len(string) > 2):
                    verse = string[0] + "." + string[1]
                else:
                    verse = string[0]
    
            return float(verse)
        else:
            
            if ("." in verses):
                string = verses.split(".")
                if (len(string) > 2):
                    verses = string[0] + "." + string[1]
                else:
                    verses = string[0]
            
            return float(verses)    
    else:
        return np.nan    

#count duplicate references,  result number is the count of item in the classification level1
def determine_aggregates(row,df,classificationLevel1):    
    target = row['Composite_reference']    
    sames = df.loc[(df['Composite_reference'] == target) & (df[classificationLevel1[0]] == row[classificationLevel1[0]]) & (df['Citation_or_allusion'] == row['Citation_or_allusion'])]    
    aggcount = len(sames)    
    return aggcount

#aggregation of sources for duplicate references
def determine_sources(row,df,classificationLevel1):   
    target = row['Composite_reference']
    sources_string = unicode(row['Work']) +"/"+ unicode(row['Work_part'])
    
    if (row['Aggregation'] > 1):
        source_string = ''
        sources = []
        sames = df.loc[(df['Composite_reference'] == target) & (df[classificationLevel1[0]] == row[classificationLevel1[0]]) & (df['Citation_or_allusion'] == row['Citation_or_allusion'])]
        sames = sames.sort_values(["ID","Work","Work_part"]) 
        for index, row in sames.iterrows():
            sources.append(unicode(row['Work']) +"/"+ unicode(row['Work_part']))
        sources_string = ', '.join(sources) 
        
        #make one of aggregates unique
        i = 0
        for index, row in sames.iterrows():
            if (i == 0):
                df.at[index, 'Aggregation'] = row['Aggregation']
            else:
                df.at[index, 'Aggregation'] = -1
            i += 1
                
    return sources_string

#new columns in dataset
for idx, df in enumerate(df_references):
    df['Book_abb'] = df.apply(lambda row: determine_book_abb(row), axis=1)
    df['Composite_reference_chapter'] = df.apply(lambda row: determine_chapter(row), axis=1)
    df['Composite_reference_verses'] = df.apply(lambda row: determine_verses(row), axis=1)
    df['Composite_reference_firstverse'] = df.apply(lambda row: determine_firstverse(row), axis=1)
    df['Aggregation'] = df.apply(lambda row: determine_aggregates(row,df,classificationLevel1), axis=1)      
    df['Reference_sources'] = df.apply(lambda row: determine_sources(row,df,classificationLevel1), axis=1)

#df_references[0]

Warning More chapter delimiters in composite reference:  1 K 12,31.13,1-3
Warning A more chapter delimiters in composite reference:  1 K 12,31.13,1-3


In [0]:
#merge of references with abbreviation on bible book

for i in range(0,len(df_references)):
    df_references[i] = df_references[i].merge(df_abb,left_on='Book_abb', right_on='Czech', how='outer')
    #print df_references[i].head(1)

    
#merge if multipledataset
if (len(df_references) > 1):
    dff = pd.concat(df_references, join="inner")
else:
    dff = df_references[0]

#print dff.head(1)

In [0]:
#test output to excel file 

dff = dff.dropna(subset=[itemReferenceColumn])
dff.sort_values([classificationLevel1[0],classificationLevel2[0],"Book_ID","Composite_reference_chapter","Composite_reference_firstverse"])
dff.to_excel(outputfilename+ ".xlsx")
files.download(outputfilename+ ".xlsx")

In [0]:
#html template

from string import Template

#OT/NT LEVEL

basic_frame = """
<html>
<head><meta charset="UTF-8"><title>Output test for Auctoritates Prototype ($outputfilename)</title>
<style>
span {display:block; }
span book {font-style:italic}
</style>
</head>
<body>
$first_frame
</body>
</html>
"""

t0 = Template(basic_frame)


In [0]:
#generate html output

#It produces indices of biblical quotes structured by biblical book
#(v pořadí určeném určeným číselnou hodnotou ve sloupci “ID” tohoto listu). 
#Under the book, the quotes are ordered by chapter and verse.


classifiers1 = dff[classificationLevel1[0]].unique().tolist()
classifiers2 = dff[classificationLevel2[0]].unique().tolist()


output = ''

for classifier1 in classifiers1:   
    if (classifier1 == np.nan):
        continue
    
    output += "<h2>" + unicode(classifier1) + "</h2><p>"         
    
    for classifier2 in classifiers2:
        
        if (str(classifier2) == "nan"):
            continue
                
        output += "</p><h3>" + classifier2 + "</h3><p>"
        
        
        #print "Comparing",classificationLevel1[0],classifier1,' a ',classificationLevel2[0],classifier2
        class_data = dff.loc[(dff[classificationLevel1[0]]==classifier1) & (dff[classificationLevel2[0]]==classifier2)]
        
        class_data = class_data.sort_values(["Book_ID","Composite_reference_chapter","Composite_reference_firstverse"])
        
        section = ''
        section_last =''
        
        for index, row in class_data.iterrows():
            reference = ''
            
            if (unicode(row['Czech_full']) == "nan"): # pass multiple and composite references [they are included in children]
                #print "Warning: beacuse of empty Czech_full passing ID: " + unicode(row['ID'])
                #print row
                continue
  
            if (row['Aggregation'] == -1): # pass references which are part of aggregation
                continue        
                
            if (row['Citation_or_allusion'] == "a"):
                before = "<span class='a'>"+leftDelimiterAllusion
                after = rightDelimiterAllusion
            else:
                before = "<span class='c'>"+leftDelimiterCitation
                after = leftDelimiterCitation
                
            
            section = row['Czech_full']
            
            if (section_last != section):              
                
                output += "</p><h4>" +section + "</h4><p>"
            
            reference += before            
                        
            if ("multiple" not in row['Composite_type']):
                reference +=  unicode(row['Composite_reference'])   
                
            if (("child" in row['Composite_type']) & (row['Composite_type'][-1] != ";")):                
                full = unicode(row['Full_reference_Czech'])
                fullparts = full.split(row['Composite_type'][-1])
                fullparts = [x.strip(' ') for x in fullparts]                
                #print row['Composite_type'][-1],fullparts, row['Composite_reference']
                fullparts.remove(row['Composite_reference'])                
                
                #to get cursive around books abb in multireference output
                partem = []
                for part in fullparts:
                    div = part.split(" ")
                    if (len(div)>3):
                        print "Warning: Strange multipart " + div                    
                    if (len(div)>2):
                        partem.append("<i>"+ div[0] +" "+ div[1]+"</i>" + div[2])
                    else:
                        partem.append("<i>"+ div[0] +"</i> "+ div[1]) 
                
                
                full = row['Composite_type'][-1]+ " " + (" " + row['Composite_type'][-1]+" ").join(partem)                    
                reference += " ["+ full +"]"
                
            reference = reference.replace(row['Book_abb'],"<i>"+unicode(row['Book_abb'])+"</i>")            
            reference += after
            reference += " .... " + unicode(row['Reference_sources']) + "\n"                    
            
            reference += "</span>"
            
            output += reference            
            section_last = row['Czech_full']

#print output

In [0]:
#d3 = {'h3_section_title':'h3 title', 'content_frame':"content"}
#html3 = t3.substitute(d3)
#d2 = {'h2_section_title':'h2 title', 'third_frame':html3}
#html2 = t2.substitute(d2)
#d1 = {'h1_section_title':'h1 title', 'second_frame':html2}
#html1 =t1.substitute(d1)
#d0 = {'first_frame':html1}
#html = t0.substitute(d0)

html = t0.substitute(first_frame=output,outputfilename=outputfilename)

#Html_file= open(outputfile + ".html","w")
#Html_file.write(html.encode("UTF-8"))
#Html_file.close()

#generate file in colab, will prompt browser download dialog
with open(outputfilename + ".html", 'w') as f:
  f.write(html.encode("UTF-8"))
files.download(outputfilename + ".html")


In [0]:
#NOT USED

#generate  output TEXTUAL

#It produces indices of biblical quotes structured by biblical book
#(v pořadí určeném určeným číselnou hodnotou ve sloupci “ID” tohoto listu). 
#Under the book, the quotes are ordered by chapter and verse.

"""
classifiers1 = dff[classificationLevel1[0]].unique().tolist()
classifiers2 = dff[classificationLevel2[0]].unique().tolist()


output = ''

for classifier1 in classifiers1:   
    if (classifier1 == np.nan):
        continue
    
    output += "\n"+ "\n" + classifier1 + "\n"     
    output += "******************************************"
    
    for classifier2 in classifiers2:
        
        if (str(classifier2) == "nan"):
            continue
                
        output += "\n" +"\n" + classifier2 + "\n"
        output += "-----------------------------------------" + "\n"
        
        #print "Comparing",classificationLevel1[0],classifier1,' a ',classificationLevel2[0],classifier2
        class_data = dff.loc[(dff[classificationLevel1[0]]==classifier1) & (dff[classificationLevel2[0]]==classifier2)]
        
        class_data = class_data.sort_values("Book_ID")
        
        section = ''
        section_last =''
        
        for index, row in class_data.iterrows():
            reference = ''
            
            if (unicode(row['Czech_full']) == "nan"):
                continue
                
            if (row['Citation_or_allusion'] == "a"):
                before = leftDelimiterAllusion
                after = rightDelimiterAllusion
            else:
                before = leftDelimiterCitation
                after = leftDelimiterCitation
                
            
            section = row['Czech_full']
            
            if (section_last != section):              
                
                output += "\n" +section + "\n"
            
            reference += before            
                        
            if ("multiple" not in row['Composite_type']):
                reference +=  unicode(row['Composite_reference'])
                
            if ("child" in row['Composite_type']):                
                full = unicode(row['Full_reference_Czech'])
                fullparts = full.split(row['Composite_type'][-1])
                fullparts = [x.strip(' ') for x in fullparts]                
                #print row['Composite_type'][-1],fullparts, row['Composite_reference']
                fullparts.remove(row['Composite_reference'])                
                full = row['Composite_type'][-1]+ " " + (" " + row['Composite_type'][-1]+" ").join(fullparts)                    
                reference += "["+ full +"]"
                        
            reference += after                              
            output += reference            
            section_last = row['Czech_full']

print output
"""


'\nclassifiers1 = dff[classificationLevel1[0]].unique().tolist()\nclassifiers2 = dff[classificationLevel2[0]].unique().tolist()\n\n\noutput = \'\'\n\nfor classifier1 in classifiers1:   \n    if (classifier1 == np.nan):\n        continue\n    \n    output += "\n"+ "\n" + classifier1 + "\n"     \n    output += "******************************************"\n    \n    for classifier2 in classifiers2:\n        \n        if (str(classifier2) == "nan"):\n            continue\n                \n        output += "\n" +"\n" + classifier2 + "\n"\n        output += "-----------------------------------------" + "\n"\n        \n        #print "Comparing",classificationLevel1[0],classifier1,\' a \',classificationLevel2[0],classifier2\n        class_data = dff.loc[(dff[classificationLevel1[0]]==classifier1) & (dff[classificationLevel2[0]]==classifier2)]\n        \n        class_data = class_data.sort_values("Book_ID")\n        \n        section = \'\'\n        section_last =\'\'\n        \n        for